In [1]:
using LinearAlgebra
using QuantumOptics
using WignerSymbols
using Plots
plotly()
using Printf 
import QuantumOptics.steadystate as steady

In [2]:
nucI=3//2
grJ=1//2
exJ=1//2

exFmax=2 #nucI+exJ
exFmin=2 #abs(nucI-exJ)
grFmax=2  #nucI+grJ
grFmin=2  #abs(nucI-grJ)

rbD1=NLevelBasis(11)  # (2Fgr+1) + (2Fex+1) + 1 states. One extra state for fly-through relaxation


function sigma(theBasis,gr,ex,F1,F2,q,mF)
    transition(theBasis,gr[(F2,mF),ex(F2,mF+q)]) 
end

function dipole(J1,J2,nucI,F1,F2,q,mF)
    wigner3j(F1,1,F2,mF,q,-mF-q)*wigner6j(J1,J2,1,F2,F1,nucI)*(-1)^(mF+J1+nucI)*sqrt((2F1+1)*(2*F2+1)*(2*J1+1))
end 


σ=[] #Array{Operator}()
dip=[]

#F1vals=[1,2]
F1vals=[2]
F2vals=[2]
qvals=[-1,1]

n2Fm=Dict{Int64,Tuple}()
Fm2n=Dict{Tuple,Int64}()
myIndex=1

for F=grFmax:-1:grFmin
    for m=F:-1:-F
        get!(n2Fm,myIndex,("ground",(F,m)))
        get!(Fm2n,("ground", (F,m)),myIndex)
        myIndex+=1
    end
end

for F=exFmax:-1:exFmin
    for m=F:-1:-F
        get!(n2Fm,myIndex,("excited", (F,m)))
        get!(Fm2n,("excited", (F,m)),myIndex)
        myIndex+=1
    end
end

for i in keys(n2Fm)
    if n2Fm[i][1]=="ground"
        F1=n2Fm[i][2][1]
        m1=n2Fm[i][2][2]
        for F2 = exFmin:exFmax
            q=+1                # sigma plus transition
            if (newN=get(Fm2n,("excited",(F2,m1+q)),0)) > 0
                push!(σ,transition(rbD1,i,newN))
                push!(dip,dipole(grJ,exJ,nucI,F1,F2,q,m1))
            end
            q=-1                # sigma minus transition
            if (newN=get(Fm2n,("excited",(F2,m1+q)),0)) > 0
                push!(σ,transition(rbD1,i,newN))
                push!(dip,dipole(grJ,exJ,nucI,F1,F2,q,m1))
            end
        end
    end
end



In [3]:
hfsEx=814.5
hfsGr=6834.682610904290
grF1=-0.625*hfsGr
grF2=0.375*hfsGr
exF1=-0.625*hfsEx
exF2=0.375*hfsEx
Δ=1.0
hfsE=Dict(
            ("ground",(2,2))=> 0, 
            ("ground",(2,1))=> 0,
            ("ground",(2,0))=> 0,
            ("ground",(2,-1))=> 0,
            ("ground",(2,-2))=> 0,
            ("excited",(2,2))=> -Δ, 
            ("excited",(2,1))=> -Δ,
            ("excited",(2,0))=> -Δ,
            ("excited",(2,-1))=> -Δ,
            ("excited",(2,-2))=> -Δ,
            )
ωLGr1=-7000  # MHz/Tesla   Larmor frequency
ωLGr2=7000 
ωLEx1=-2300  # MHz/Tesla   Larmor frequency
ωLEx2=2300   
larmor=Dict(
            ("ground",(2,2))=> 2.0*ωLGr2, 
            ("ground",(2,1))=> 1.0*ωLGr2,
            ("ground",(2,0))=> 0*ωLGr2,
            ("ground",(2,-1))=> -1.0*ωLGr2,
            ("ground",(2,-2))=> -2.0*ωLGr2,
            ("excited",(2,2))=> 2.0*ωLEx2, 
            ("excited",(2,1))=> 1.0*ωLEx2,
            ("excited",(2,0))=> 0*ωLEx2,
            ("excited",(2,-1))=> -1.0*ωLEx2,
            ("excited",(2,-2))=> -2.0*ωLEx2,
            )
Ω=1.0       # Rabi frequency
Γ=1.0       # Natual linewidth
γ=0.05       # Transit relaxation
flyThrough = 11 #17 # Level for transit (fly-through) relaxation
nGround    = 5  # number of levels in the ground state 



proj=[]
detune=[]
ωL=[]
H=identityoperator(σ[1])*0
J=[]

for i=1:length(n2Fm)
    push!(proj,transition(rbD1,i,i))
    push!(ωL,larmor[n2Fm[i]])
    push!(detune,hfsE[n2Fm[i]])
 #   H += ( detune[i] + ωL[i]*0.001 )*proj[i] # diagonal part of Hamiltonian
    push!(J,sqrt(γ)*transition(rbD1,flyThrough,i))                            # Jump for fly-through relaxation out
    if n2Fm[i][1] == "ground"
        push!(J,sqrt(γ/nGround)*transition(rbD1,i,flyThrough))                    # Jump for fly-through relaxation in 
    end
    for j=1:length(n2Fm)
        if n2Fm[i][1]=="excited" && n2Fm[j][1]=="ground"
             push!(J,sqrt(Γ/nGround)*transition(rbD1,j,i))
        end
    end
end




In [4]:
Bmin=-1.0e-3 #-5.0e-6
Bmax=1.0e-3 #5.0e-6
Bstep=1.0e-5 #1.0e-7
ΩR=1.0       # Rabi frequency
Γ=1.0       # Natual linewidth
γ=0.05       # Transit relaxation
Bvalues=[]
absorb=[]
rotation=[]
popEx=[]
popGr=[]

for B=Bmin:Bstep:Bmax
    H=identityoperator(σ[1])*0
    for i=1:length(n2Fm)
        H += (  detune[i] + ωL[i]*B )*proj[i]
#        H += (  ωL[i]*B )*proj[i]
    end
    for i=1:length(σ)
        H+= ΩR*dip[i]*(σ[i]+dagger(σ[i]))   # Hamiltonian
    end
    ρ_eig=steady.eigenvector(DenseOperator(H),J)
    obs1=3*(Γ/ΩR)*real(expect(σ[1],ρ_eig) 
                        +expect(σ[2],ρ_eig) 
                        +expect(σ[3],ρ_eig) 
                        +expect(σ[4],ρ_eig) 
                        - expect(σ[5],ρ_eig) 
                        -expect(σ[6],ρ_eig) 
                        -expect(σ[7],ρ_eig) 
                        -expect(σ[8],ρ_eig) 
                        )
    obs2=3*(Γ/ΩR)*real(expect(σ[1],ρ_eig) 
                        +expect(σ[2],ρ_eig) 
                        +expect(σ[3],ρ_eig) 
                        +expect(σ[4],ρ_eig) 
                        +expect(σ[5],ρ_eig) 
                        +expect(σ[6],ρ_eig) 
                        +expect(σ[7],ρ_eig) 
                        +expect(σ[8],ρ_eig) 
                        )
    obs3=2*real(
        expect(proj[6],ρ_eig) 
        +expect(proj[7],ρ_eig) 
        +expect(proj[8],ρ_eig) 
        +expect(proj[9],ρ_eig) 
        +expect(proj[10],ρ_eig) 
        )
        obs4=2*real(
            expect(proj[1],ρ_eig) 
            +expect(proj[2],ρ_eig) 
            +expect(proj[3],ρ_eig) 
            +expect(proj[4],ρ_eig) 
            +expect(proj[5],ρ_eig) 
            )
    push!(Bvalues,B)
    push!(absorb,obs1)
    push!(rotation,obs2)
    push!(popEx,obs3)
    push!(popGr,obs4)
end

In [5]:
plot1=Plots.plot(Bvalues,popEx,ylabel="Excited State Population")
plot2=Plots.plot(Bvalues,popGr,ylabel="Ground State Population")
plot3=Plots.plot(Bvalues,absorb)
plot4=Plots.plot(Bvalues,rotation)
Plots.plot(plot1, plot2, plot3, plot4, layout = (2, 2), legend = false)

<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
Δmin=-500 
Δmax=500
Δstep=5
B=5.0e-3 
ΩR=10.0       # Rabi frequency
Γ=1.0       # Natual linewidth
γ=0.05       # Transit relaxation
Δvalues=[]
absorb=[]
rotation=[]
popEx=[]
popGr=[]

for Δ=Δmin:Δstep:Δmax   # loop over detuning
    hfsE=Dict(
            ("ground",(2,2))=> 0, 
            ("ground",(2,1))=> 0,
            ("ground",(2,0))=> 0,
            ("ground",(2,-1))=> 0,
            ("ground",(2,-2))=> 0,
            ("excited",(2,2))=> -Δ, 
            ("excited",(2,1))=> -Δ,
            ("excited",(2,0))=> -Δ,
            ("excited",(2,-1))=> -Δ,
            ("excited",(2,-2))=> -Δ,
            )
    detune=[]
    [ push!(detune,hfsE[n2Fm[i]]) for i in 1:length(n2Fm) ]   # fill detuning array (very inefficient approach)
    H=identityoperator(σ[1])*0
    for i=1:length(n2Fm)
        H += (  detune[i] + ωL[i]*B )*proj[i]
    end
    for i=1:length(σ)
        H+= ΩR*dip[i]*(σ[i]+dagger(σ[i]))   # Hamiltonian
    end
    ρ_eig=steady.eigenvector(DenseOperator(H),J)
    obs1=3*(Γ/ΩR)*real(expect(σ[1],ρ_eig) 
                        +expect(σ[2],ρ_eig) 
                        +expect(σ[3],ρ_eig) 
                        +expect(σ[4],ρ_eig) 
                        - expect(σ[5],ρ_eig) 
                        -expect(σ[6],ρ_eig) 
                        -expect(σ[7],ρ_eig) 
                        -expect(σ[8],ρ_eig) 
                        )
    obs2=3*(Γ/ΩR)*real(expect(σ[1],ρ_eig) 
                        +expect(σ[2],ρ_eig) 
                        +expect(σ[3],ρ_eig) 
                        +expect(σ[4],ρ_eig) 
                        +expect(σ[5],ρ_eig) 
                        +expect(σ[6],ρ_eig) 
                        +expect(σ[7],ρ_eig) 
                        +expect(σ[8],ρ_eig) 
                        )
    obs3=2*real(
        expect(proj[6],ρ_eig) 
        +expect(proj[7],ρ_eig) 
        +expect(proj[8],ρ_eig) 
        +expect(proj[9],ρ_eig) 
        +expect(proj[10],ρ_eig) 
        )
        obs4=2*real(
            expect(proj[1],ρ_eig) 
            +expect(proj[2],ρ_eig) 
            +expect(proj[3],ρ_eig) 
            +expect(proj[4],ρ_eig) 
            +expect(proj[5],ρ_eig) 
            )
    push!(Δvalues,Δ)
    push!(absorb,obs1)
    push!(rotation,obs2)
    push!(popEx,obs3)
    push!(popGr,obs4)
end

In [7]:
plot5=Plots.plot(Δvalues,popEx)
plot6=Plots.plot(Δvalues,popGr)
plot7=Plots.plot(Δvalues,absorb)
plot8=Plots.plot(Δvalues,rotation)
Plots.plot(plot5, plot6, plot7, plot8, layout = (2, 2), legend = false)

<!DOCTYPE html>
 
 
 Plots.jl